<a href="https://colab.research.google.com/github/sebatlab/Antaki2021/blob/main/Antaki2021_R_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Table of Contents
-------------------
* [Genetic Correlations](#genetic_correlations)
* [Clinical Analysis by Sex](#clinical_analysis_by_sex)
* [Genetic Correlations by Sex](#genetic_cor_sex)
* [Variance Explained by Genetic Risk Factors](#variance_explained_by_genetic_risk_factors)
* [Paternal and Maternal Age Correlations with De Novo Mutations](#paternal_maternal_age_correlations)

In [ ]:
# Install packages
install.packages("gplots")
install.packages("basicTrendline")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘bitops’, ‘gtools’, ‘caTools’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘investr’




In [ ]:
# Attaching packages
library(gplots)
library(basicTrendline)


Attaching package: ‘gplots’


The following object is masked from ‘package:stats’:

    lowess




<a name="genetic_correlations"></a>
## Genetic Correlations


In [ ]:
spark.rv.prs<-read.delim("antaki2021_full_genetic_table.tsv", header = TRUE,sep = "\t")
spark.rv.prs$all_dn<-spark.rv.prs$snv_dn_lof+spark.rv.prs$snv_dn_mis
male<-spark.rv.prs$sex==0
female<-spark.rv.prs$sex==1
case<-spark.rv.prs$phen==1
cont<-spark.rv.prs$phen==0
phen<-spark.rv.prs$phen
sex<-spark.rv.prs$sex

library(gplots)
library(basicTrendline)
##################correlation of rare and common
#quick t tests of combined PRS
spark.rv.prs$all_dn_t<-spark.rv.prs$snv_dn_lof+spark.rv.prs$snv_dn_mis+spark.rv.prs$snv_lof_mt+spark.rv.prs$snv_lof_pt

rv.case.m<-spark.rv.prs$all_dn>0 & case & male
nrv.case.m<-spark.rv.prs$all_dn_==0 & case & male
rv.cont.m<-spark.rv.prs$all_dn>0 & cont & male
nrv.cont.m<-spark.rv.prs$all_dn==0 & cont & male

rv.case.f<-spark.rv.prs$all_dn>0 & case & female
nrv.case.f<-spark.rv.prs$all_dn==0 & case & female
rv.cont.f<-spark.rv.prs$all_dn>0 & cont & female
nrv.cont.f<-spark.rv.prs$all_dn==0 & cont & female

dnlof.case.m<-spark.rv.prs$snv_dn_lof >0 & case & male
ndnlof.case.m<-spark.rv.prs$snv_dn_lof==0 & case & male
dnlof.cont.m<-spark.rv.prs$snv_dn_lof >0 & cont & male
ndnlof.cont.m<-spark.rv.prs$snv_dn_lof==0 & cont & male

dnlof.case.f<-spark.rv.prs$snv_dn_lof >0 & case & female
ndnlof.case.f<-spark.rv.prs$snv_dn_lof==0 & case & female
dnlof.cont.f<-spark.rv.prs$snv_dn_lof >0 & cont & female
ndnlof.cont.f<-spark.rv.prs$snv_dn_lof==0 & cont & female

#t-tests of liability threshold all de novo
t.test(spark.rv.prs$ps_combined_score[rv.case.m],spark.rv.prs$ps_combined_score[nrv.case.m]) #male case p = 0.020
t.test(spark.rv.prs$ps_combined_score[rv.case.f],spark.rv.prs$ps_combined_score[nrv.case.f]) #female case p = 1

t.test(spark.rv.prs$ps_combined_score[rv.cont.m],spark.rv.prs$ps_combined_score[nrv.cont.m]) #male cont p = 0.39
t.test(spark.rv.prs$ps_combined_score[rv.cont.f],spark.rv.prs$ps_combined_score[nrv.cont.f]) #female cont p = 86

#t-tests of liability threshold de novo lof
t.test(spark.rv.prs$ps_combined_score[dnlof.case.m],spark.rv.prs$ps_combined_score[ndnlof.case.m]) #male case p = 0.042
t.test(spark.rv.prs$ps_combined_score[dnlof.case.f],spark.rv.prs$ps_combined_score[ndnlof.case.f]) #female case p = 0.44

t.test(spark.rv.prs$ps_combined_score[dnlof.cont.m],spark.rv.prs$ps_combined_score[ndnlof.cont.m]) #male cont p = 0.035
t.test(spark.rv.prs$ps_combined_score[dnlof.cont.f],spark.rv.prs$ps_combined_score[ndnlof.cont.f]) #female cont p = 0.32


#regression tests
sex.cor<-spark.rv.prs[,c("rare_combined_score","ps_combined_score")]    
model<-lm(sex.cor$ps_combined_score~sex.cor$rare_combined_score+sex+phen)
summary(model) #p = 0.079

int.model<-lm(sex.cor$ps_combined_score~sex.cor$rare_combined_score*sex+phen)
summary(int.model) #p = 0.047

sex.cor<-spark.rv.prs[male,c("rare_combined_score","ps_combined_score")]    
model<-lm(sex.cor$ps_combined_score~sex.cor$rare_combined_score+phen[male])
summary(model) #p = 0.014

sex.cor<-spark.rv.prs[female,c("rare_combined_score","ps_combined_score")]    
model<-lm(sex.cor$ps_combined_score~sex.cor$rare_combined_score+phen[female])
summary(model) #p = 0.68

trendline(spark.rv.prs$rare_combined_score[male & case],spark.rv.prs$ps_combined_score[male & case])
trendline(spark.rv.prs$rare_combined_score[male & cont],spark.rv.prs$ps_combined_score[male & cont])

########################### all pairwise correlations
gen.names<-c("snv_dn_lof" , "snv_lof_t", "rare_combined_score" , "ptdt_prs_asd" , "ptdt_prs_ea" ,"ptdt_prs_scz","ps_combined_score")
x.names<-vector(length=(length(gen.names))^2)
y.names<-vector(length=(length(x.names)))
est.col<-vector(length=(length(x.names)))
x.p.col<-vector(length=(length(x.names)))
r.col<-vector(length=(length(x.names)))
pear.col<-vector(length=(length(x.names)))
sex.p.col<-vector(length=(length(x.names)))

#males/females combined
temp<-spark.rv.prs
PC1<-temp$PC1
PC2<-temp$PC2
PC3<-temp$PC3
PC4<-temp$PC4
PC5<-temp$PC5
PC6<-temp$PC6
PC7<-temp$PC7
PC8<-temp$PC8
PC9<-temp$PC9
PC10<-temp$PC10
phen<-temp$phen
sex<-temp$sex


gen.cor<-temp[,gen.names]
n<-1
for (i in 1:ncol(gen.cor))
{
  
  for(j in 1:ncol(gen.cor))
  {
    cat(n)
    y<-gen.cor[[i]]
    x<-gen.cor[[j]]
    model<-lm(y~x+sex+phen+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
    n.model<-lm(y~sex+phen+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
    est<-signif(summary(model)$coefficients[2,1],digits=2)
    x.pval<-signif(summary(model)$coefficients[2,4],digits=2)
    r.sq<-signif(summary(model)$r.squared-summary(n.model)$r.squared,digits=2)
    pearson.cc<-signif(cor(y,x, use="pairwise.complete.obs"),digits = 2)
    
    #test sex interaction
    int.model<-lm(y~x*sex+phen+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
    sex.pval<-signif(summary(int.model)$coefficients[15,4],digits=2)
    
    y.names[n]<-gen.names[i]
    x.names[n]<-gen.names[j]
    est.col[n]<-est
    x.p.col[n]<-x.pval
    r.col[n]<-r.sq
    pear.col[n]<-pearson.cc
    sex.p.col[n]<-sex.pval
    n<-n+1
  }
}
comb.gen.cor.result<-data.frame(y.names,x.names,est.col,x.p.col,r.col,pear.col,sex.p.col)

write.csv(comb.gen.cor.result,"asd_genetic_correlations_combined.csv")





#males first
temp<-spark.rv.prs[male,]
PC1<-temp$PC1
PC2<-temp$PC2
PC3<-temp$PC3
PC4<-temp$PC4
PC5<-temp$PC5
PC6<-temp$PC6
PC7<-temp$PC7
PC8<-temp$PC8
PC9<-temp$PC9
PC10<-temp$PC10
phen<-temp$phen


gen.cor<-temp[,gen.names]
n<-1
for (i in 1:ncol(gen.cor))
{

  for(j in 1:ncol(gen.cor))
  {
    cat(n)
    y<-gen.cor[[i]]
    x<-gen.cor[[j]]
    model<-lm(y~x+phen+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
    n.model<-lm(y~phen+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
    est<-signif(summary(model)$coefficients[2,1],digits=2)
    x.pval<-signif(summary(model)$coefficients[2,4],digits=2)
    r.sq<-signif(summary(model)$r.squared-summary(n.model)$r.squared,digits=2)
    pearson.cc<-signif(cor(y,x, use="pairwise.complete.obs"),digits = 2)
    
    y.names[n]<-gen.names[i]
    x.names[n]<-gen.names[j]
    est.col[n]<-est
    x.p.col[n]<-x.pval
    r.col[n]<-r.sq
    pear.col[n]<-pearson.cc
    n<-n+1
  }
}
male.gen.cor<-data.frame(y.names,x.names,est.col,x.p.col,r.col,pear.col)

#now females
temp<-spark.rv.prs[female,]
PC1<-temp$PC1
PC2<-temp$PC2
PC3<-temp$PC3
PC4<-temp$PC4
PC5<-temp$PC5
PC6<-temp$PC6
PC7<-temp$PC7
PC8<-temp$PC8
PC9<-temp$PC9
PC10<-temp$PC10
phen<-temp$phen


gen.cor<-temp[,gen.names]
n<-1
for (i in 1:ncol(gen.cor))
{
  
  for(j in 1:ncol(gen.cor))
  {
    cat(n)
    y<-gen.cor[[i]]
    x<-gen.cor[[j]]
    model<-lm(y~x+phen+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
    n.model<-lm(y~phen+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
    est<-signif(summary(model)$coefficients[2,1],digits=2)
    x.pval<-signif(summary(model)$coefficients[2,4],digits=2)
    r.sq<-signif(summary(model)$r.squared-summary(n.model)$r.squared,digits=2)
    pearson.cc<-signif(cor(y,x, use="pairwise.complete.obs"),digits = 2)
    
    y.names[n]<-gen.names[i]
    x.names[n]<-gen.names[j]
    est.col[n]<-est
    x.p.col[n]<-x.pval
    r.col[n]<-r.sq
    pear.col[n]<-pearson.cc
    n<-n+1
  }
}
female.gen.cor<-data.frame(y.names,x.names,est.col,x.p.col,r.col,pear.col)

#compile final result
gen.cor.result<-data.frame(male.gen.cor,female.gen.cor)

write.csv(gen.cor.result,"asd_genetic_correlations_by_sex.csv")



#first test all rare x all common
#close but no cigar in the combined cases and controls p = 0.08
summary(lm(y.prs~x.rare+sex+phen))
cor(y.prs,x.rare)

#sex interaction is significant (P = 0.011)

int.model<-lm(y.prs~x.rare*sex)
summary(int.model)

#Negative correlation of rare and common is significant in males (p = 0.014)
summary(lm(y.prs[male]~x.rare[male]+phen[male]))
cor(y.prs[male],x.rare[male])

#No correlation of rare and common in females (p = 0.68)
summary(lm(y.prs[female]~x.rare[female]+phen[female]))
cor(y.prs[female],x.rare[female])


############## common and rare negative correlation does not stratify by case status (stronger in case but NS in both)

summary(lm(y.prs[case]~x.rare[case]+sex[case]))
cor(y.prs[case],x.rare[case])

summary(lm(y.prs[cont]~x.rare[cont]+sex[cont]))
cor(y.prs[cont],x.rare[cont])

summary(lm(y.prs[male & case]~x.rare[male & case])) #p = 0.035
cor(y.prs[male & case],x.rare[male & case])

summary(lm(y.prs[male & cont]~x.rare[male & cont])) #p = 0.18
cor(y.prs[male & cont],x.rare[male & cont])

<a name="clinical_analysis_by_sex"></a>
## Clinical Analysis by Sex

In [ ]:
# RV PRS and model residuals
spark.rv.prs <- read.delim("antaki2021_full_genetic_table.tsv", header = TRUE,sep="\t")

# Sum "all" categories combined and sum the parents
spark.rv.prs$snv_lof_t <- spark.rv.prs$snv_lof_mt + spark.rv.prs$snv_lof_pt
spark.rv.prs$snv_lof_n <- spark.rv.prs$snv_lof_mn + spark.rv.prs$snv_lof_pn
spark.rv.prs$all_dn_lof <- spark.rv.prs$snv_dn_lof 
spark.rv.prs$all_dn <- spark.rv.prs$snv_dn_lof + spark.rv.prs$snv_dn_mis
spark.rv.prs$all_mt <- spark.rv.prs$snv_lof_mt 
spark.rv.prs$all_mn <- spark.rv.prs$snv_lof_mn 
spark.rv.prs$all_pt <- spark.rv.prs$snv_lof_pt 
spark.rv.prs$all_pn <- spark.rv.prs$snv_lof_pn 
spark.rv.prs$all_t <- spark.rv.prs$all_pt + spark.rv.prs$all_mt
spark.rv.prs$all_n <- spark.rv.prs$all_pn + spark.rv.prs$all_mn
spark.rv.prs$all_dn_t <- spark.rv.prs$all_t + spark.rv.prs$all_dn

# Clinical tables for SSC
ssc.ids <- read.table("SSC_IDs_Sanders.txt", header = TRUE, sep="\t")
ssc.age <- read.csv("SSC_Age_of_Subjects_&_ParentalAgeAtBirth.csv", header=TRUE) 
names(ssc.age)[2] <- "individual"
ssc.age <- ssc.age[,c("individual","subject_age")]

# DCDQ
dcdq.ssc <- read.csv("dcdq.csv",header = TRUE)
temp1 <- ssc.ids[match(dcdq.ssc$individual,ssc.ids$individual),]
temp2 <- ssc.age[match(dcdq.ssc$individual,ssc.age$individual),]
dcdq.ssc <- data.frame(temp1,temp2, dcdq.ssc)
dcdq.ssc$total.z <- (dcdq.ssc$total-mean(dcdq.ssc$total,na.rm=TRUE))/sd(dcdq.ssc$total,na.rm=TRUE)

# SCQ
scq.ssc.p <- read.csv("scq_life.csv",header = TRUE)
scq.ssc.s <- read.csv("scq_life.csv",header = TRUE)
scq.ssc <- rbind.data.frame(scq.ssc.p,scq.ssc.s)
temp1 <- ssc.ids[match(scq.ssc$individual,ssc.ids$individual),]
temp2 <- ssc.age[match(scq.ssc$individual,ssc.age$individual),]
scq.ssc <- data.frame(temp1,temp2,scq.ssc)
scq.ssc$summary_score.z <- (scq.ssc$summary_score-mean(scq.ssc$summary_score,na.rm=TRUE))/sd(scq.ssc$summary_score,na.rm=TRUE)
scq.ssc.temp <- scq.ssc[,c("SS_RUID","subject_age","summary_score.z")]

# SRS
father.srs <- read.csv("srs_score_father.csv", header = TRUE)
temp1 <- ssc.ids[match(father.srs$individual,ssc.ids$individual),]
temp2 <- ssc.age[match(father.srs$individual,ssc.age$individual),]
father.srs <- data.frame(temp1,temp2, father.srs)

mother.srs <- read.csv("srs_score_mother.csv", header = TRUE)
temp1 <- ssc.ids[match(mother.srs$individual,ssc.ids$individual),]
temp2 <- ssc.age[match(mother.srs$individual,ssc.age$individual),]
mother.srs <- data.frame(temp1,temp2, mother.srs)

proband.srs <- read.csv("srs_score_proband.csv", header = TRUE)
temp1 <- ssc.ids[match(proband.srs$individual,ssc.ids$individual),]
temp2 <- ssc.age[match(proband.srs$individual,ssc.age$individual),]
proband.srs <- data.frame(temp1,temp2, proband.srs)
father.total <- father.srs[match(proband.srs$family,father.srs$family),"total"]
mother.total <- mother.srs[match(proband.srs$family,mother.srs$family),"total"]
proband.srs <- data.frame(proband.srs,mother.total,father.total)
proband.srs <- proband.srs[,c("SS_RUID", "individual", "family", "relation", "collection", "subject_age", "total", "mother.total", "father.total")]
temp3 <- spark.rv.prs[match(proband.srs$SS_RUID,spark.rv.prs$iid),]
proband.srs <- data.frame(proband.srs,temp3)

sibling.srs <- read.csv("srs_score_sib.csv", header = TRUE)
temp1 <- ssc.ids[match(sibling.srs$individual,ssc.ids$individual),]
temp2 <- ssc.age[match(sibling.srs$individual,ssc.age$individual),]
sibling.srs <- data.frame(temp1,temp2,sibling.srs)
father.total <- father.srs[match(sibling.srs$family,father.srs$family),"total"]
mother.total <- mother.srs[match(sibling.srs$family,mother.srs$family),"total"]
sibling.srs <- data.frame(sibling.srs,mother.total,father.total)
sibling.srs <- sibling.srs[,c("SS_RUID", "individual", "family", "relation", "collection", "subject_age", "total", "mother.total", "father.total")]
temp3 <- spark.rv.prs[match(sibling.srs$SS_RUID,spark.rv.prs$iid),]
sibling.srs <- data.frame(sibling.srs,temp3)

ssc.srs <- rbind(proband.srs,sibling.srs)
ssc.srs <- ssc.srs[!is.na(ssc.srs$fid),]

# Master table
temp <- rbind(proband.srs,sibling.srs)
temp <- temp[,c(1,6,7,8,9)]
names(temp) <- c("srs.individual","srs.child.age","srs.child","srs.mother","srs.father")
temp2 <- temp[match(master.phen$iid,temp$srs.individual),]

# Vineland
proband.vine <- read.csv("vineland_comp_proband.csv", header = TRUE)
temp1 <- ssc.ids[match(proband.vine$individual,ssc.ids$individual),]
temp2 <- ssc.age[match(proband.vine$individual,ssc.age$individual),]
proband.vine <- data.frame(temp1,temp2, proband.vine)

sibling.vine <- read.csv("vineland_comp_sib.csv", header = TRUE)
temp1 <- ssc.ids[match(sibling.vine$individual,ssc.ids$individual),]
temp2 <- ssc.age[match(sibling.vine$individual,ssc.age$individual),]
sibling.vine <- data.frame(temp1,temp2, sibling.vine)

ssc.vine <- rbind(proband.vine,sibling.vine)
ssc.vine <- ssc.vine[,setdiff(1:ncol(ssc.vine),2:9)] #removing extra columns
temp <- spark.rv.prs[match(ssc.vine$SS_RUID,spark.rv.prs$iid),]
ssc.vine <- data.frame(ssc.vine,temp)
ssc.vine <- ssc.vine[!is.na(ssc.vine$fid),]

write.csv(ssc.vine, "vine_ssc.csv")

# Add to master table
temp <- rbind(proband.vine,sibling.vine)
temp <- temp[,setdiff(1:ncol(temp),2:9)]
temp <- temp[,c(1,2,4)]
names(temp) <- c("vineland.individual", "vineland.age", "vineland.composite.score")
temp2 <- temp[match(master.phen$iid,temp$vineland.individual),]
master.phen <- data.frame(master.phen,temp2)

# BAPQ
bapq.father <- read.csv("bapq_score_father.csv", header = TRUE)
temp1 <- ssc.ids[match(bapq.father$individual,ssc.ids$individual),]
temp2 <- ssc.age[match(bapq.father$individual,ssc.age$individual),]
bapq.father <- data.frame(temp1,temp2, bapq.father)

bapq.mother <- read.csv("bapq_score_mother.csv", header = TRUE)
temp1 <- ssc.ids[match(bapq.mother$individual,ssc.ids$individual),]
temp2 <- ssc.age[match(bapq.mother$individual,ssc.age$individual),]
bapq.mother <- data.frame(temp1,temp2, bapq.mother)

# Final BAPQ
temp1 <- spark.rv.prs[which(spark.rv.prs$phen==1),]

names(bapq.father)[1:2] <- c("individual","individual.3")
temp2 <- temp1[match(bapq.father$family,temp1$fid),]
bapq.father <- data.frame(bapq.father,temp2)
bapq.father <- bapq.father[!is.na(bapq.father$fid),]

names(bapq.mother)[1:2] <- c("individual","individual.3")
temp2 <- temp1[match(bapq.mother$family,temp1$fid),]
bapq.mother <- data.frame(bapq.mother,temp2)
bapq.mother <- bapq.mother[!is.na(bapq.mother$fid),]

write.csv(bapq.mother, "bapq_mother_ssc.csv")
write.csv(bapq.father, "bapq_father_ssc.csv")

temp<-bapq.father

# Add BAPQ to master table
bapq.father<-read.csv("bapq_score_father.csv", header = TRUE)
temp1<-ssc.ids[match(bapq.father$individual,ssc.ids$individual),]
temp2<-ssc.age[match(bapq.father$individual,ssc.age$individual),]
bapq.father<-data.frame(temp1,temp2, bapq.father)
temp.father<-bapq.father[,c(3,10,12)]
names(temp.father)<-c("father.family","father.age","father.bapq")

bapq.mother<-read.csv("bapq_score_mother.csv", header = TRUE)
temp1<-ssc.ids[match(bapq.mother$individual,ssc.ids$individual),]
temp2<-ssc.age[match(bapq.mother$individual,ssc.age$individual),]
bapq.mother<-data.frame(temp1,temp2, bapq.mother)
temp.mother<-bapq.mother[,c(3,10,12)]
names(temp.mother)<-c("mother.family","mother.age","mother.bapq")
temp3<-temp.father[match(temp.mother$mother.family,temp.father$father.family),]
bapq.mother.father<-data.frame(temp.mother,temp3)

# Master table
temp2 <- temp.mother[match(master.phen$fid,temp.mother$mother.family),]
temp3 <- temp.father[match(master.phen$fid,temp.father$father.family),]
master.phen <- data.frame(master.phen,temp2,temp3)

# Quick check on whether mother and father bapq are correlated
x <- bapq.mother.father$mother.bapq
y <- bapq.mother.father$father.bapq
summary(lm(y~x))

# Clinical tables for SPARK
scq.spark <- read.csv("scq_summary.csv", header = TRUE)
scq.spark$summary_score.z <- (scq.spark$summary_score-mean(scq.spark$summary_score,na.rm=TRUE))/sd(scq.spark$summary_score,na.rm=TRUE)
scq.spark.temp <- scq.spark[,c("subject_sp_id","age_at_eval_months","summary_score.z")]

dcdq.spark <- read.csv("dcdq_summary.csv", header = TRUE)
dcdq.spark$total.z <- (dcdq.spark$total-mean(dcdq.spark$total,na.rm=TRUE))/sd(dcdq.spark$total,na.rm=TRUE)
dcdq.ssc.temp <- dcdq.ssc[,c("SS_RUID","subject_age","total.z")]
dcdq.spark.temp <- dcdq.spark[,c("subject_sp_id","age_at_eval_months","total.z")]

# Final SCQ table
names(scq.spark.temp)[1]<-"individual"
names(scq.ssc.temp)<-names(scq.spark.temp)
scq.ssc.spark<-rbind.data.frame(scq.ssc.temp,scq.spark.temp)
temp<-spark.rv.prs[match(scq.ssc.spark$individual,spark.rv.prs$iid),]
scq.ssc.spark<-data.frame(scq.ssc.spark,temp)
scq.ssc.spark<-scq.ssc.spark[!is.na(scq.ssc.spark$fid),]


# Final RBS table

rbs.ssc<-read.csv("rbs_r_summary.csv", header = TRUE)
temp1<-ssc.ids[match(rbs.ssc$individual,ssc.ids$individual),]
temp2<-ssc.age[match(rbs.ssc$individual,ssc.age$individual),]
rbs.ssc<-data.frame(temp1,temp2, rbs.ssc)
rbs.ssc$summary_score.z<-(rbs.ssc$overall_score-mean(rbs.ssc$overall_score,na.rm=TRUE))/sd(rbs.ssc$overall_score,na.rm=TRUE)
rbs.spark<-read.csv("C:\\Users\\jsebat\\sebatlab Dropbox\\Autism\\SPARK_Phenotype_Data_Version2\\rbs_r_summary.csv", header = TRUE)
rbs.spark$summary_score.z<-(rbs.spark$total_final_score-mean(rbs.spark$total_final_score,na.rm=TRUE))/sd(rbs.spark$total_final_score,na.rm=TRUE)

names(rbs.spark)[1]<-"individual"
rbs.spark<-rbs.spark[,c("individual","summary_score.z")]
rbs.ssc<-rbs.ssc[,c("SS_RUID","summary_score.z")]
names(rbs.ssc)[1]<-"individual"
rbs.ssc.spark<-rbind.data.frame(rbs.ssc,rbs.spark)
temp<-spark.rv.prs[match(rbs.ssc.spark$individual,spark.rv.prs$iid),]
rbs.ssc.spark<-data.frame(rbs.ssc.spark,temp)
rbs.ssc.spark<-rbs.ssc.spark[!is.na(rbs.ssc.spark$fid),]

write.csv(rbs.ssc.spark, "C:\\Users\\jsebat\\sebatlab Dropbox\\Manuscripts\\Antaki_Autism_2020\\PhenotypeAnalysis\\DataTables\\rbs_ssc_spark.csv")

# Add to master table
temp<-rbind.data.frame(rbs.ssc,rbs.spark)
names(temp)<-c("rbs.individual", "rbs.z")
temp2<-temp[match(master.phen$iid,temp$rbs.individual),]
master.phen<-data.frame(master.phen,temp2)


# Final SCQ table
names(scq.spark.temp)[1]<-"individual"
names(scq.ssc.temp)<-names(scq.spark.temp)
scq.ssc.spark<-rbind.data.frame(scq.ssc.temp,scq.spark.temp)
temp<-spark.rv.prs[match(scq.ssc.spark$individual,spark.rv.prs$iid),]
scq.ssc.spark<-data.frame(scq.ssc.spark,temp)
scq.ssc.spark<-scq.ssc.spark[!is.na(scq.ssc.spark$fid),]

write.csv(scq.ssc.spark, "C:\\Users\\jsebat\\sebatlab Dropbox\\Manuscripts\\Antaki_Autism_2020\\PhenotypeAnalysis\\DataTables\\scq_ssc_spark.csv")

# Add to master table
temp<-rbind.data.frame(scq.ssc.temp,scq.spark.temp)
names(temp)<-c("scq.individual","age_at_scq", "scq.z")
temp2<-temp[match(master.phen$iid,temp$scq.individual),]
master.phen<-data.frame(master.phen,temp2)

# Final DCDQ
names(dcdq.spark.temp)[1]<-"individual"
names(dcdq.ssc.temp)<-names(dcdq.spark.temp)
dcdq.ssc.spark<-rbind.data.frame(dcdq.ssc.temp, dcdq.spark.temp)
temp<-spark.rv.prs[match(dcdq.ssc.spark$individual,spark.rv.prs$iid),]
dcdq.ssc.spark<-data.frame(dcdq.ssc.spark,temp)
dcdq.ssc.spark<-dcdq.ssc.spark[!is.na(dcdq.ssc.spark$fid),]

write.csv(dcdq.ssc.spark, "C:\\Users\\jsebat\\sebatlab Dropbox\\Manuscripts\\Antaki_Autism_2020\\PhenotypeAnalysis\\DataTables\\dcdq_ssc_spark.csv")


# Add to master table
temp<-rbind.data.frame(dcdq.ssc.temp, dcdq.spark.temp)
names(temp)<-c("dcdq.individual","age_at_dcdq", "dcdq.z")
temp2<-temp[match(master.phen$iid,temp$dcdq.individual),]
master.phen<-data.frame(master.phen,temp2)

#bghx #father/mother education SSC_background_hx, Spark bghx

# Import the bghx tables. Also add a column of NAs to be used for missing fields
bghx.spark.pro<-read.csv("C:\\Users\\jsebat\\sebatlab Dropbox\\Autism\\SPARK_Phenotype_Data_Version2\\bghx_child.csv",header = TRUE)
bghx.spark.sib<-read.csv("C:\\Users\\jsebat\\sebatlab Dropbox\\Autism\\SPARK_Phenotype_Data_Version2\\bghx_sibling.csv",header = TRUE)
bghx.spark.sib$na.col<-rep(NA,nrow(bghx.spark.sib))
spark.sib.nacol<-ncol(bghx.spark.sib)
bghx.ssc<-read.csv("C:\\Users\\jsebat\\sebatlab Dropbox\\Autism\\SSC_Version_15_Phenotype_Data_Set_9\\Proband Data\\ssc_background_hx.csv",header = TRUE)
temp1<-ssc.ids[match(bghx.ssc$individual,ssc.ids$individual),]
bghx.ssc$individual<-temp1$SS_RUID
bghx.ssc$na.col<-rep(NA,nrow(bghx.ssc))
ssc.nacol<-ncol(bghx.ssc)

# Import the column merge file and then create a vector of column selections of all the matching columns and NAs for missing fields
bghx.col.merge<-read.csv("C:\\Users\\jsebat\\sebatlab Dropbox\\Autism\\Autism_Illumina_WGS\\RV_PRS_Analysis_Jun2020\\bghx_col_merge.csv",header = TRUE)

# Vectors of column selections
spark.sib.col<-bghx.col.merge$spark_sib_col
spark.sib.col[is.na(spark.sib.col)]<-spark.sib.nacol
ssc.col<-bghx.col.merge$ssc_pro_col
ssc.col[is.na(ssc.col)]<-ssc.nacol

# Create bghx table for each group with standardized columns
temp.spark.pro<-bghx.spark.pro[,bghx.col.merge$spark_pro_col]
temp.names<-names(temp.spark.pro)
temp.names[1]<-"iid"
names(temp.spark.pro)<-temp.names

temp.spark.sib<-bghx.spark.sib[,spark.sib.col]
names(temp.spark.sib)<-temp.names

temp.ssc<-bghx.ssc[,ssc.col]
names(temp.ssc)<-temp.names


bghx.spark.ssc<-rbind(temp.spark.pro,temp.spark.sib,temp.ssc)

# Fixing the education fields

edu1<-c("less-ninth","up-ninth","did_not_attend_high_school")
edu2<-c("some-hs", "some_high_school")
edu3<-c("ged", "ged_diploma")
edu4<-c("high-school", "high_school_graduate")
edu5<-c("trade_school")
edu6<-c("some-college","some_college")
edu7<-c("associate", "associate_degree")
edu8<-c("baccalaureate", "baccalaureate_degree")
edu9<-c("graduate","graduate_or_professional_degree") 

mother.edu<-rep(NA,length=nrow(bghx.spark.ssc))
mother.edu[is.element(bghx.spark.ssc$mother_highest_education,edu1)]<-1
mother.edu[is.element(bghx.spark.ssc$mother_highest_education,edu2)]<-2
mother.edu[is.element(bghx.spark.ssc$mother_highest_education,edu3)]<-3
mother.edu[is.element(bghx.spark.ssc$mother_highest_education,edu4)]<-4
mother.edu[is.element(bghx.spark.ssc$mother_highest_education,edu5)]<-5
mother.edu[is.element(bghx.spark.ssc$mother_highest_education,edu6)]<-6
mother.edu[is.element(bghx.spark.ssc$mother_highest_education,edu7)]<-7
mother.edu[is.element(bghx.spark.ssc$mother_highest_education,edu8)]<-8
mother.edu[is.element(bghx.spark.ssc$mother_highest_education,edu9)]<-9

father.edu<-rep(NA,length=nrow(bghx.spark.ssc))
father.edu[is.element(bghx.spark.ssc$father_highest_education,edu1)]<-1
father.edu[is.element(bghx.spark.ssc$father_highest_education,edu2)]<-2
father.edu[is.element(bghx.spark.ssc$father_highest_education,edu3)]<-3
father.edu[is.element(bghx.spark.ssc$father_highest_education,edu4)]<-4
father.edu[is.element(bghx.spark.ssc$father_highest_education,edu5)]<-5
father.edu[is.element(bghx.spark.ssc$father_highest_education,edu6)]<-6
father.edu[is.element(bghx.spark.ssc$father_highest_education,edu7)]<-7
father.edu[is.element(bghx.spark.ssc$father_highest_education,edu8)]<-8
father.edu[is.element(bghx.spark.ssc$father_highest_education,edu9)]<-9

bghx.spark.ssc$mother_highest_education<-mother.edu
bghx.spark.ssc$father_highest_education<-father.edu

trendline(mother.edu,father.edu,ePos.x="topright")

# Final bghx table
temp<-spark.rv.prs[match(bghx.spark.ssc$iid,spark.rv.prs$iid,),]
bghx.spark.ssc<-data.frame(bghx.spark.ssc,temp)
bghx.spark.ssc<-bghx.spark.ssc[!is.na(bghx.spark.ssc$fid),]

write.csv(bghx.spark.ssc, "C:\\Users\\jsebat\\sebatlab Dropbox\\Manuscripts\\Antaki_Autism_2020\\PhenotypeAnalysis\\DataTables\\bghx_ssc_spark.csv")

# Add to master table
temp<-bghx.spark.ssc
temp<-temp[,c("iid.1","mother_highest_education","father_highest_education")]
names(temp)<-c("EA.individual","mother.EA","father.EA")
temp2<-temp[match(master.phen$iid,temp$EA.individual),]
master.phen<-data.frame(master.phen,temp2)

# Add parental age to master table
temp<-parental.age[,c("iid","FATHER_AGE", "MOTHER_AGE")]
names(temp)<-c("parental.age.child","father.age.at.birth","mother.age.at.birth")
temp2<-temp[match(master.phen$iid,temp$parental.age.child),]
master.phen<-data.frame(master.phen,temp2)

master.phen.2<-master.phen[,c(1:8,58:85)]
temp.names<- c(  "fid"     ,                 "iid"       ,               "phen"        ,             "sex"      ,               
                 "cohort"             ,      "duo"        ,              "family"      ,             "is_eur"     ,             
                "srs.individual"   ,        "srs.child.age"    ,        "srs.child"   ,             "srs.mother"     ,         
                "srs.father"      ,         "vineland.individual"   ,   "child.vineland.age"    ,         "child.vineland.composite.score",
                 "mother.family"   ,         "mother.age"       ,        "mother.bapq"      ,        "father.family"    ,       
                 "father.age"       ,        "father.bapq"       ,       "rbs.individual"    ,       "child.rbs.z"      ,             
                 "scq.individual"   ,        "age_at_scq"     ,          "child.scq.z"      ,              "dcdq.individual"   ,      
                 "age_at_dcdq"      ,        "child.dcdq.z"      ,             "EA.individual"  ,          "mother.EA"       ,        
                "father.EA"       ,         "parental.age.child"   ,    "father.age.at.birth"   ,   "mother.age.at.birth" )
names(master.phen.2)<-temp.names

temp.names2<- c(  "fid"     ,                 "iid"       ,               "phen"        ,             "sex"      ,               
                 "cohort"             ,      "duo"        ,              "family"      ,             "is_eur"     ,             
                  "srs.child.age"    ,        "srs.child"   ,           "child.vineland.age"    ,         "child.vineland.composite.score",
                   "child.rbs.z"      ,             
                  "age_at_scq"     ,          "child.scq.z"      ,      
                 "age_at_dcdq"      ,        "child.dcdq.z"      ,          "father.age.at.birth"   ,   "mother.age.at.birth", 
                 "mother.EA"       ,       "father.EA"       ,  "srs.mother"     ,         
                 "srs.father"      ,  "mother.bapq"      ,          "father.bapq" )

master.phen.2<-master.phen.2[,temp.names2]

write.csv(master.phen.2,"C:\\Users\\jsebat\\sebatlab Dropbox\\Autism\\Autism_Illumina_WGS\\RV_PRS_Analysis_Jun2020\\master_phenotype_table.csv")


temp.3<-master.phen.2[match(spark.rv.prs$iid,master.phen.2$iid),]
master.phen.3<-data.frame(spark.rv.prs,temp.3)


# Start analysis with table master.phen.3
# First create a function that runs the regression on both sexes and outputs the result
gen.phen.1<-function(master.temp,phen.col,age.col)
{
#define the sexes
male<-master.temp$sex=="Male"
female<-master.temp$sex=="Female"
  
#Test combined sample
sex<-master.temp$sex
PC1<-master.temp$PC1
PC2<-master.temp$PC2
PC3<-master.temp$PC3
PC4<-master.temp$PC4
PC5<-master.temp$PC5
PC6<-master.temp$PC6
PC7<-master.temp$PC7
PC8<-master.temp$PC8
PC9<-master.temp$PC9
PC10<-master.temp$PC10

#retreive phen data
y<-master.temp[,phen.col]
age<-master.temp[,age.col]


#retreive gen data
x.dnlof<-master.temp[,"all_dn_lof"]
x.dn<-master.temp[,"all_dn"]
x.inh<-master.temp[,"all_t"]
x.ps.asd<-master.temp[,"prs_asd"]
x.ps.sz<-master.temp[,"prs_scz"]
x.ps.ea<-master.temp[,"prs_ea"]
x.ps.iq<-master.temp[,"prs_int"]

result.dnlof<-lr.1(y,x.dnlof,sex,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
result.dn<-lr.1(y,x.dn,sex,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
result.inh<-lr.1(y,x.inh,sex,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
result.ps.asd<-lr.1(y,x.ps.asd,sex,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
result.ps.sz<-lr.1(y,x.ps.sz,sex,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
result.ps.ea<-lr.1(y,x.ps.ea,sex,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
result.ps.iq<-lr.1(y,x.ps.iq,sex,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)

comb.result<-rbind(result.dnlof,result.dn,result.inh,result.ps.asd,result.ps.sz,result.ps.ea,result.ps.iq)
variant.type<-c("dnlof","dn ms + lof","inh rv", "ps.asd","ps.sz","ps.ea","ps.iq")
group<-rep("combined",length(variant.type))
comb.result<-cbind(group,variant.type,comb.result)

#################Test males

PC1<-master.temp$PC1[male]
PC2<-master.temp$PC2[male]
PC3<-master.temp$PC3[male]
PC4<-master.temp$PC4[male]
PC5<-master.temp$PC5[male]
PC6<-master.temp$PC6[male]
PC7<-master.temp$PC7[male]
PC8<-master.temp$PC8[male]
PC9<-master.temp$PC9[male]
PC10<-master.temp$PC10[male]

#retreive phen data
y<-master.temp[male,phen.col]
age<-master.temp[male,age.col]


#retreive gen data
x.dnlof<-master.temp[male,"all_dn_lof"]
x.dn<-master.temp[male,"all_dn"]
x.inh<-master.temp[male,"all_t"]
x.ps.asd<-master.temp[male,"prs_asd"]
x.ps.sz<-master.temp[male,"prs_scz"]
x.ps.ea<-master.temp[male,"prs_ea"]
x.ps.iq<-master.temp[male,"prs_int"]

result.dnlof<-lr.2(y,x.dnlof,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
result.dn<-lr.2(y,x.dn,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
result.inh<-lr.2(y,x.inh,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
result.ps.asd<-lr.2(y,x.ps.asd,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
result.ps.sz<-lr.2(y,x.ps.sz,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
result.ps.ea<-lr.2(y,x.ps.ea,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
result.ps.iq<-lr.2(y,x.ps.iq,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)

male.result<-rbind(result.dnlof,result.dn,result.inh,result.ps.asd,result.ps.sz,result.ps.ea,result.ps.iq)
variant.type<-c("dnlof","dn ms + lof","inh rv", "ps.asd","ps.sz","ps.ea","ps.iq")
group<-rep("male",length(variant.type))
male.result<-cbind(group,variant.type,male.result)

#####Test females ######################



PC1<-master.temp$PC1[female]
PC2<-master.temp$PC2[female]
PC3<-master.temp$PC3[female]
PC4<-master.temp$PC4[female]
PC5<-master.temp$PC5[female]
PC6<-master.temp$PC6[female]
PC7<-master.temp$PC7[female]
PC8<-master.temp$PC8[female]
PC9<-master.temp$PC9[female]
PC10<-master.temp$PC10[female]

#retreive phen data
y<-master.temp[female,phen.col]
age<-master.temp[female,age.col]


#retreive gen data
x.dnlof<-master.temp[female,"all_dn_lof"]
x.dn<-master.temp[female,"all_dn"]
x.inh<-master.temp[female,"all_t"]
x.ps.asd<-master.temp[female,"prs_asd"]
x.ps.sz<-master.temp[female,"prs_scz"]
x.ps.ea<-master.temp[female,"prs_ea"]
x.ps.iq<-master.temp[female,"prs_int"]

result.dnlof<-lr.2(y,x.dnlof,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
result.dn<-lr.2(y,x.dn,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
result.inh<-lr.2(y,x.inh,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
result.ps.asd<-lr.2(y,x.ps.asd,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
result.ps.sz<-lr.2(y,x.ps.sz,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
result.ps.ea<-lr.2(y,x.ps.ea,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
result.ps.iq<-lr.2(y,x.ps.iq,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)

female.result<-rbind(result.dnlof,result.dn,result.inh,result.ps.asd,result.ps.sz,result.ps.ea,result.ps.iq)
variant.type<-c("dnlof","dn ms + lof","inh rv", "ps.asd","ps.sz","ps.ea","ps.iq")
group<-rep("female",length(variant.type))
female.result<-cbind(group,variant.type,female.result)

final.result<-rbind(female.result,male.result,comb.result)
final.result
}



#linear regression script 1 tests sex interaction
lr.1<-function(y,x,sex,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
{
  model<-lm(y~x+sex+age+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
  n.model<-lm(y~sex+age+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
  est<-signif(summary(model)$coefficients[2,1],digits=2)
  x.pval<-signif(summary(model)$coefficients[2,4],digits=2)
  r.sq<-signif(summary(model)$r.squared-summary(n.model)$r.squared,digits=2)
  pearson.cc<-signif(cor(y,x, use="pairwise.complete.obs"),digits = 2)
  
  #test sex interaction
  int.model<-lm(y~x*sex+age+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
  sex.pval<-signif(summary(int.model)$coefficients[15,4],digits=2)
  
  result<-cbind(r.sq,est,pearson.cc,x.pval,sex.pval)
  result
}

###############################################################################33
#Then we have a function that doesn't test the sex interaction
##############################################################################

gen.phen.2<-function(master.temp,phen.col,age.col)
{
  #retreive covariates
  PC1<-master.temp$PC1
  PC2<-master.temp$PC2
  PC3<-master.temp$PC3
  PC4<-master.temp$PC4
  PC5<-master.temp$PC5
  PC6<-master.temp$PC6
  PC7<-master.temp$PC7
  PC8<-master.temp$PC8
  PC9<-master.temp$PC9
  PC10<-master.temp$PC10
  
  #retreive phen data
  y<-master.temp[,phen.col]
  age<-master.temp[,age.col]
  
  #retreive gen data
  x.dnlof<-master.temp[,"all_dn_lof"]
  x.dn<-master.temp[,"all_dn"]
  x.inh<-master.temp[,"all_t"]
  x.ps.asd<-master.temp[,"prs_asd"]
  x.ps.sz<-master.temp[,"prs_scz"]
  x.ps.ea<-master.temp[,"prs_ea"]
  x.ps.iq<-master.temp[,"prs_int"]
  
  result.dnlof<-lr.2(y,x.dnlof,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.dn<-lr.2(y,x.dn,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.inh<-lr.2(y,x.inh,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.ps.asd<-lr.2(y,x.ps.asd,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.ps.sz<-lr.2(y,x.ps.sz,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.ps.ea<-lr.2(y,x.ps.ea,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.ps.iq<-lr.2(y,x.ps.iq,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  
  final.result<-rbind(result.dnlof,result.dn,result.inh,result.ps.asd,result.ps.sz,result.ps.ea,result.ps.iq)
  variant.type<-c("dnlof","dn ms + lof","inh rv", "ps.asd","ps.sz","ps.ea","ps.iq")
  final.result<-cbind(variant.type,final.result)
  final.result
}



#linear regression script no sex
lr.2<-function(y,x,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
{
model<-lm(y~x+age+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
n.model<-lm(y~age+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
est<-signif(summary(model)$coefficients[2,1],digits=2)
x.pval<-signif(summary(model)$coefficients[2,4],digits=2)
r.sq<-signif(summary(model)$r.squared-summary(n.model)$r.squared,digits=2)
pearson.cc<-signif(cor(y,x, use="pairwise.complete.obs"),digits = 2)
#test sex interaction
sex.pval<-NA
result<-cbind(r.sq,est,pearson.cc,x.pval,sex.pval)
result
}


##################################################################
# Then we have a regression for testing the effects of parents
#################################################################

#Ftests phenotypes in parents and tests sex interaction
gen.phen.3<-function(master.temp,phen.col.f,phen.col.m)
{
  #retrieve father data
  ###########################
  sexfather<-rep(1,nrow(master.temp))
  PC1f<-master.temp$PC1_PAT
  PC2f<-master.temp$PC2_PAT
  PC3f<-master.temp$PC3_PAT
  PC4f<-master.temp$PC4_PAT
  PC5f<-master.temp$PC5_PAT
  PC6f<-master.temp$PC6_PAT
  PC7f<-master.temp$PC7_PAT
  PC8f<-master.temp$PC8_PAT
  PC9f<-master.temp$PC9_PAT
  PC10f<-master.temp$PC10_PAT
  
  #retreive phen data
  yf<-master.temp[,phen.col.f]
  
  
  #retreive gen data
  f.dnlof<-master.temp[,"all_dn_lof"]
  f.dn<-master.temp[,"all_dn"]
  f.inh<-master.temp$all_pt + master.temp$all_pn
  f.ps.asd<-master.temp$prs_asd_pat
  f.ps.sz<-master.temp$prs_scz_pat
  f.ps.ea<-master.temp$prs_ea_pat
  f.ps.iq<-master.temp$prs_int_pat
  
  #retrieve mother data
  ###########################
  
  sexmother<-rep(0,nrow(master.temp))
  PC1m<-master.temp$PC1_MAT
  PC2m<-master.temp$PC2_MAT
  PC3m<-master.temp$PC3_MAT
  PC4m<-master.temp$PC4_MAT
  PC5m<-master.temp$PC5_MAT
  PC6m<-master.temp$PC6_MAT
  PC7m<-master.temp$PC7_MAT
  PC8m<-master.temp$PC8_MAT
  PC9m<-master.temp$PC9_MAT
  PC10m<-master.temp$PC10_MAT
  
  #retreive phen data
  ym<-master.temp[,phen.col.m]
  
  #retreive gen data
  m.dnlof<-master.temp[,"all_dn_lof"]
  m.dn<-master.temp[,"all_dn"]
  m.inh<-master.temp$all_mt + master.temp$all_mn
  m.ps.asd<-master.temp$prs_asd_mat
  m.ps.sz<-master.temp$prs_scz_mat
  m.ps.ea<-master.temp$prs_ea_mat
  m.ps.iq<-master.temp$prs_int_mat
  
  #merge father and mother variables
  sex<-c(sexfather,sexmother)
  PC1<-c(PC1f,PC1m)
  PC2<-c(PC2f,PC2m)
  PC3<-c(PC3f,PC3m)
  PC4<-c(PC4f,PC4m)
  PC5<-c(PC5f,PC5m)
  PC6<-c(PC6f,PC6m)
  PC7<-c(PC7f,PC7m)
  PC8<-c(PC8f,PC8m)
  PC9<-c(PC9f,PC9m)
  PC10<-c(PC10f,PC10m)
  x.dnlof<-c(f.dnlof,m.dnlof)
  x.dn<-c(f.dn,m.dn)
  x.inh<-c(f.inh,m.inh)
  x.ps.asd<-c(f.ps.asd,m.ps.asd)
  x.ps.sz<-c(f.ps.sz,m.ps.sz)
  x.ps.ea<-c(f.ps.ea,m.ps.ea)
  x.ps.iq<-c(f.ps.iq,m.ps.iq)
  y<-c(yf,ym)
  
  result.dnlof<-lr.3(y,x.dnlof,sex,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.dn<-lr.3(y,x.dn,sex,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.inh<-lr.3(y, x.inh, sex, PC1, PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.ps.asd<-lr.3(y,x.ps.asd,sex,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.ps.sz<-lr.3(y,x.ps.sz,sex,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.ps.ea<-lr.3(y,x.ps.ea,sex,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.ps.iq<-lr.3(y,x.ps.iq,sex,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  
  #combined reult
  comb.result<-rbind(result.dnlof,result.dn,result.inh,result.ps.asd,result.ps.sz,result.ps.ea,result.ps.iq)
  variant.type<-c("dnlof","dn ms + lof","inh rv", "ps.asd","ps.sz","ps.ea","ps.iq")
  group<-rep("parents",length=length(variant.type))
  comb.result<-cbind(group,variant.type,comb.result)
  
  
  #father result
  result.dnlof<-lr.4(yf,f.dnlof,PC1f,PC2f,PC3f,PC4f,PC5f,PC6f,PC7f,PC8f,PC9f,PC10f)
  result.dn<-lr.4(yf,f.dn,PC1f,PC2f,PC3f,PC4f,PC5f,PC6f,PC7f,PC8f,PC9f,PC10f)
  result.inh<-lr.4(yf, f.inh, PC1f,PC2f,PC3f,PC4f,PC5f,PC6f,PC7f,PC8f,PC9f,PC10f)
  result.ps.asd<-lr.4(yf,f.ps.asd,PC1f,PC2f,PC3f,PC4f,PC5f,PC6f,PC7f,PC8f,PC9f,PC10f)
  result.ps.sz<-lr.4(yf,f.ps.sz,PC1f,PC2f,PC3f,PC4f,PC5f,PC6f,PC7f,PC8f,PC9f,PC10f)
  result.ps.ea<-lr.4(yf,f.ps.ea,PC1f,PC2f,PC3f,PC4f,PC5f,PC6f,PC7f,PC8f,PC9f,PC10f)
  result.ps.iq<-lr.4(yf,f.ps.iq,PC1f,PC2f,PC3f,PC4f,PC5f,PC6f,PC7f,PC8f,PC9f,PC10f)
  
  f.result<-rbind(result.dnlof,result.dn,result.inh,result.ps.asd,result.ps.sz,result.ps.ea,result.ps.iq)
  variant.type<-c("dnlof","dn ms + lof","inh rv", "ps.asd","ps.sz","ps.ea","ps.iq")
  group<-rep("father",length=length(variant.type))
  f.result<-cbind(group,variant.type,f.result)
  
  #mother result
  result.dnlof<-lr.4(ym,m.dnlof,PC1m,PC2m,PC3m,PC4m,PC5m,PC6m,PC7m,PC8m,PC9m,PC10m)
  result.dn<-lr.4(ym,m.dn,PC1m,PC2m,PC3m,PC4m,PC5m,PC6m,PC7m,PC8m,PC9m,PC10m)
  result.inh<-lr.4(ym, m.inh, PC1m,PC2m,PC3m,PC4m,PC5m,PC6m,PC7m,PC8m,PC9m,PC10m)
  result.ps.asd<-lr.4(ym,m.ps.asd,PC1m,PC2m,PC3m,PC4m,PC5m,PC6m,PC7m,PC8m,PC9m,PC10m)
  result.ps.sz<-lr.4(ym,m.ps.sz,PC1m,PC2m,PC3m,PC4m,PC5m,PC6m,PC7m,PC8m,PC9m,PC10m)
  result.ps.ea<-lr.4(ym,m.ps.ea,PC1m,PC2m,PC3m,PC4m,PC5m,PC6m,PC7m,PC8m,PC9m,PC10m)
  result.ps.iq<-lr.4(ym,m.ps.iq,PC1m,PC2m,PC3m,PC4m,PC5m,PC6m,PC7m,PC8m,PC9m,PC10m)
  
  m.result<-rbind(result.dnlof,result.dn,result.inh,result.ps.asd,result.ps.sz,result.ps.ea,result.ps.iq)
  variant.type<-c("dnlof","dn ms + lof","inh rv", "ps.asd","ps.sz","ps.ea","ps.iq")
  group<-rep("mother",length=length(variant.type))
  m.result<-cbind(group,variant.type,m.result)
  
  #output final result
  final.result<-rbind(m.result,f.result,comb.result)
  final.result
}


#linear regression script 3 tests phenotypes in parents and tests sex interaction
lr.3<-function(y,x,sex,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
{
  model<-lm(y~x+sex+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
  n.model<-lm(y~sex+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
  est<-signif(summary(model)$coefficients[2,1],digits=2)
  x.pval<-signif(summary(model)$coefficients[2,4],digits=2)
  r.sq<-signif(summary(model)$r.squared-summary(n.model)$r.squared,digits=2)
  pearson.cc<-signif(cor(y,x, use="pairwise.complete.obs"),digits = 2)
  
  #test sex interaction
  int.model<-lm(y~x*sex++PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
  sex.pval<-signif(summary(int.model)$coefficients[14,4],digits=2)
  
  result<-cbind(r.sq,est,pearson.cc,x.pval,sex.pval)
  result
}

#linear regression script 4 tests phenotypes in mother or father without sex variable
lr.4<-function(y,x,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
{
  model<-lm(y~x+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
  n.model<-lm(y~PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
  est<-signif(summary(model)$coefficients[2,1],digits=2)
  x.pval<-signif(summary(model)$coefficients[2,4],digits=2)
  r.sq<-signif(summary(model)$r.squared-summary(n.model)$r.squared,digits=2)
  pearson.cc<-signif(cor(y,x, use="pairwise.complete.obs"),digits = 2)
  
  #test sex interaction
  sex.pval<-NA
  
  result<-cbind(r.sq,est,pearson.cc,x.pval,sex.pval)
  result
}


####################################################
# Now we test each phenotype for given set of trios
#####################################################

######################3
# offspring
####################3

cont<-master.phen.3$phen=="CON"
case<-master.phen.3$phen=="ASD"
eur<-master.phen.3$is_eur==1


#Define the groups that we will test
#all families
master.temp.case<-master.phen.3[case,]
master.temp.cont<-master.phen.3[cont,]

#excluding non-eur ancestry
master.temp.case.eur<-master.phen.3[case & eur,]
master.temp.cont.eur<-master.phen.3[cont & eur,]


#Vineland

phen.col<-"child.vineland.composite.score"
age.col<-"child.vineland.age"

vine.case<-gen.phen.1(master.temp = master.temp.case, phen.col=phen.col, age.col=age.col)
vine.cont<-gen.phen.1(master.temp = master.temp.cont, phen.col=phen.col, age.col=age.col)
vine.case.eur<-gen.phen.1(master.temp = master.temp.case.eur, phen.col=phen.col, age.col=age.col)
vine.cont.eur<-gen.phen.1(master.temp = master.temp.cont.eur, phen.col=phen.col, age.col=age.col)

phenotype<-rep("vineland",nrow(vine.case))
vine.case<-cbind(phenotype,vine.case)
vine.case.eur<-cbind(phenotype,vine.case.eur)

phenotype<-rep("vineland",nrow(vine.cont))
vine.cont<-cbind(phenotype,vine.cont)
vine.cont.eur<-cbind(phenotype,vine.cont.eur)

#SRS
phen.col<-"srs.child"
age.col<-"srs.child.age"

srs.case<-gen.phen.1(master.temp = master.temp.case, phen.col=phen.col, age.col=age.col)
srs.case.eur<-gen.phen.1(master.temp = master.temp.case.eur, phen.col=phen.col, age.col=age.col)

srs.cont<-gen.phen.1(master.temp = master.temp.cont, phen.col=phen.col, age.col=age.col)
srs.cont.eur<-gen.phen.1(master.temp = master.temp.cont.eur, phen.col=phen.col, age.col=age.col)


phenotype<-rep("SRS",nrow(srs.case))
srs.case<-cbind(phenotype,srs.case)
srs.case.eur<-cbind(phenotype,srs.case.eur)

phenotype<-rep("SRS",nrow(srs.cont))
srs.cont<-cbind(phenotype,srs.cont)
srs.cont.eur<-cbind(phenotype,srs.cont.eur)


#SCQ
phen.col<-"child.scq.z"
age.col<-"age_at_scq"

scq.case<-gen.phen.1(master.temp = master.temp.case, phen.col=phen.col, age.col=age.col)
scq.cont<-gen.phen.1(master.temp = master.temp.cont, phen.col=phen.col, age.col=age.col)
scq.case.eur<-gen.phen.1(master.temp = master.temp.case.eur, phen.col=phen.col, age.col=age.col)
scq.cont.eur<-gen.phen.1(master.temp = master.temp.cont.eur, phen.col=phen.col, age.col=age.col)

phenotype<-rep("SCQ",nrow(scq.case))
scq.case<-cbind(phenotype,scq.case)
scq.case.eur<-cbind(phenotype,scq.case.eur)
phenotype<-rep("SCQ",nrow(scq.cont))
scq.cont<-cbind(phenotype,scq.cont)
scq.cont.eur<-cbind(phenotype,scq.cont.eur)


#DCDQ
phen.col<-"child.dcdq.z"
age.col<-"age_at_dcdq"

dcdq.case<-gen.phen.1(master.temp = master.temp.case, phen.col=phen.col, age.col=age.col)
dcdq.case.eur<-gen.phen.1(master.temp = master.temp.case.eur, phen.col=phen.col, age.col=age.col)

phenotype<-rep("DCDQ",nrow(dcdq.case))
dcdq.case<-cbind(phenotype,dcdq.case)
dcdq.case.eur<-cbind(phenotype,dcdq.case.eur)


#rbs
phen.col<-"child.rbs.z"
age.col<-"age_at_scq"

rbs.case<-gen.phen.1(master.temp = master.temp.case, phen.col=phen.col, age.col=age.col)
rbs.case.eur<-gen.phen.1(master.temp = master.temp.case.eur, phen.col=phen.col, age.col=age.col)

phenotype<-rep("RBS",nrow(rbs.case))
rbs.case<-cbind(phenotype,rbs.case)
rbs.case.eur<-cbind(phenotype,rbs.case.eur)



#################33
#Parents
##############################3

cont<-master.phen.3$phen=="CON"
case<-master.phen.3$phen=="ASD"
eur<-master.phen.3$is_eur==1


#Define the groups that we will test
master.temp.case<-master.phen.3[case,]
master.temp.case.eur<-master.phen.3[case & eur,]

##############BAPQ

#define the phenotypes
phen.col.f<-"father.bapq"
phen.col.m<-"mother.bapq"

bapq.parents<-gen.phen.3(master.temp.case,phen.col.f,phen.col.m)
bapq.parents.eur<-gen.phen.3(master.temp.case.eur,phen.col.f,phen.col.m)

phenotype<-rep("BAPQ",nrow(bapq.parents))
bapq.parents<-cbind(phenotype,bapq.parents)
bapq.parents.eur<-cbind(phenotype,bapq.parents.eur)



###################SRS
#define the phenotypes
phen.col.f<-"srs.father"
phen.col.m<-"srs.mother"

srs.parents<-gen.phen.3(master.temp.case,phen.col.f,phen.col.m)
srs.parents.eur<-gen.phen.3(master.temp.case.eur,phen.col.f,phen.col.m)


phenotype<-rep("SRS",nrow(srs.parents))
srs.parents<-cbind(phenotype,srs.parents)
srs.parents.eur<-cbind(phenotype,srs.parents.eur)


###################Parental Age
#define the phenotypes
phen.col.f<-"father.age.at.birth"
phen.col.m<-"mother.age.at.birth"

parental.age<-gen.phen.3(master.temp.case,phen.col.f,phen.col.m)
parental.age.eur<-gen.phen.3(master.temp.case.eur,phen.col.f,phen.col.m)


phenotype<-rep("Parental Age",nrow(parental.age))
parental.age<-cbind(phenotype,parental.age)
parental.age.eur<-cbind(phenotype,parental.age.eur)

###################EA
#define the phenotypes
phen.col.f<-"father.EA"
phen.col.m<-"mother.EA"

parental.ea<-gen.phen.3(master.temp.case,phen.col.f,phen.col.m)
parental.ea.eur<-gen.phen.3(master.temp.case.eur,phen.col.f,phen.col.m)

phenotype<-rep("Parental EA",nrow(parental.ea))
parental.ea<-cbind(phenotype,parental.ea)
parental.ea.eur<-cbind(phenotype,parental.ea.eur)


clin.results.ssc.spark<-rbind(srs.case,scq.case,vine.case,dcdq.case,rbs.case,srs.cont,
                              scq.cont,vine.cont,bapq.parents,srs.parents,parental.age,parental.ea)
clin.results.ssc.spark.eur<-rbind(srs.case.eur,scq.case.eur,vine.case.eur,dcdq.case.eur,rbs.case.eur,srs.cont.eur,
                                  scq.cont.eur,vine.cont.eur,bapq.parents.eur,srs.parents.eur,parental.age.eur,parental.ea.eur)
write.csv(clin.results.ssc.spark,"C:\\Users\\jsebat\\sebatlab Dropbox\\Autism\\Autism_Illumina_WGS\\RV_PRS_Analysis_Jun2020\\clin_results_ssc_spark.csv")
write.csv(clin.results.ssc.spark.eur,"C:\\Users\\jsebat\\sebatlab Dropbox\\Autism\\Autism_Illumina_WGS\\RV_PRS_Analysis_Jun2020\\clin_results_ssc_spark_eur.csv")



<a name="genetic_cor_sex"></a>
## Genetic Correlations by Sex
### Figure 3b
-------------------------------

In [ ]:
options(stringsAsFactors = FALSE);
df = read.csv("/home/dantakli/asd/model/SupplementaryTable2.tsv",sep="\t")
df$dnAll = df$dnLoF+df$dnMIS
df$cohort_covar <- factor(df$Cohort)
df <- df[df$EUR.Ancestry==1,]
vars<-c(
    "dnMIS" ,"dnLoF","dnAll",
    "inhLoF", "Rare.Combined.Score" , 
    "pTDT.ASD.Dev" , "pTDT.SCZ.Dev" ,"pTDT.EA.Dev",
    "PS.Combined.Score"
);
cols <- c("cohort_covar","Sex","Phenotype","PC1","PC2","PC3","PC4","PC5","PC6","PC7","PC8","PC9","PC10",vars)
df<-na.omit(df[cols])
print(nrow(df))
###########################################
fit_model<- function(df,x,y,add_sex=FALSE){
    Y<- df[[x]]
    X<- df[[y]]
    PC1<-df$PC1
    PC2<-df$PC2
    PC3<-df$PC3
    PC4<-df$PC4
    PC5<-df$PC5
    PC6<-df$PC6
    PC7<-df$PC7
    PC8<-df$PC8
    PC9<-df$PC9
    PC10<-df$PC10
    phen<-df$Phenotype
    cohort_covar<-df$cohort_covar
    sex<-df$Sex
    sex_pv<-NA    if (add_sex==TRUE){
        mod<-lm(Y~X+sex+phen+cohort_covar+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
        nul<-lm(Y~sex+phen+cohort_covar+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
        #sex interaction
        interaction_mod<-lm(Y~X*sex+phen+cohort_covar+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
        sex_pv<-signif(summary(interaction_mod)$coefficients[17,4],digits=2)
    } else {
        mod<-lm(Y~X+phen+cohort_covar+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
        nul<-lm(Y~phen+cohort_covar+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
    }
    # slope of fit of X to Y
    est<-signif(summary(mod)$coefficients[2,1],digits=2)
    xpv<-signif(summary(mod)$coefficients[2,4],digits=2)
    #r2 is the difference of the mod with X and without X
    r2<-signif(summary(mod)$r.squared-summary(nul)$r.squared,digits=2)
    pearsons_cor<-signif(cor(Y,X, use="pairwise.complete.obs"),digits=2)    if (add_sex==TRUE){
        result = paste(est,xpv,r2,pearsons_cor,sex_pv,sep="\t")
    } else {
        result = paste(est,xpv,r2,pearsons_cor,sep="\t")
    }
    #sprintf("%f\t%f\t%f\t%f\t%f",est,xpv,r2,pearson_cor,sex_pv)
    return(result)
}
###########################################
header = paste("y","x",
               "male_est","male_pval","male_r2","male_cor",
               "female_est","female_pval","female_r2","female_cor",
               "combined_est","combined_pval","combined_r2","combined_cor",
               "sex_interaction_pval",
               sep="\t"
              );
output=vector(length = 1+(length(vars)^2)-(length(vars)))
output[1]<-header
i=2
for (x in vars)
{
  for(y in vars)
  {
    if (x==y){ next; } 
    com_results <- fit_model(df,x,y,add_sex=TRUE)
    mal_results <- fit_model(df[df$Sex==0,],x,y,add_sex=FALSE)
    fem_results <- fit_model(df[df$Sex==1,],x,y,add_sex=FALSE)
    # Y - X - mal
    output[i] <- paste(x,y,mal_results,fem_results,com_results,sep="\t")
    i=i+1;
  }
}out = "asd_genetic_correlations_by_sex_final.tsv"
ofh<-file(out)
writeLines(output,ofh)
close(ofh)print(">>>DONE<<<")

<a name="variance_explained_by_genetic_risk_factors"></a>
## Variance Explained by Genetic Risk Factors

In [ ]:
# all_combined

library(fmsb)
library(lmtest)
library(boot)
options(stringsAsFactors = FALSE);

incremental_r2 <- function(formula, data, indices) {
    d <- data[indices,]
    fit <- glm(formula, data=d, family=binomial)
    fit_alt  <- glm(d$phen~d$sex+d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10+d$snv_dn_lof+d$snv_dn_mis+d$snv_lof_t+d$prs_asd+d$prs_ea+d$prs_int+d$prs_scz, data=d, family=binomial)
    fit_null <- glm(d$phen~d$sex+d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10, data=d, family=binomial)
    return(NagelkerkeR2(fit_alt)$R2 - NagelkerkeR2(fit_null)$R2)
}

incremental_r2_sex <- function(formula, data, indices) {
    d <- data[indices,]
    fit <- glm(formula, data=d, family=binomial)
    fit_alt  <- glm(d$phen~d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10+d$snv_dn_lof+d$snv_dn_mis+d$snv_lof_t+d$prs_asd+d$prs_ea+d$prs_int+d$prs_scz, data=d, family=binomial)
    fit_null <- glm(d$phen~d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10, data=d, family=binomial)
    return(NagelkerkeR2(fit_alt)$R2 - NagelkerkeR2(fit_null)$R2)
}

runlr <- function(d) {
    fit_alt  <- glm(d$phen~d$sex+d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10+d$snv_dn_lof+d$snv_dn_mis+d$snv_lof_t+d$prs_asd+d$prs_ea+d$prs_int+d$prs_scz, data=d, family=binomial)
    fit_null <- glm(d$phen~d$sex+d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10, data=d, family=binomial)
    return(lrtest(fit_alt,fit_null))   
}

runlr_sex <- function(d) {
    fit_alt  <- glm(d$phen~d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10+d$snv_dn_lof+d$snv_dn_mis+d$snv_lof_t+d$prs_asd+d$prs_ea+d$prs_int+d$prs_scz, data=d, family=binomial)
    fit_null <- glm(d$phen~d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10, data=d, family=binomial)
    return(lrtest(fit_alt,fit_null))   
}

get_risk_stats <-function(df_both,df_male,df_female,filename){
    df_both_stats <- get_stats(df=df_both,statfunc=incremental_r2,"both","tmp_both.csv",runlr)
    df_male_stats <- get_stats(df=df_male,statfunc=incremental_r2_sex,"male","tmp_male.csv",runlr_sex)
    df_female_stats <- get_stats(df=df_female,statfunc=incremental_r2_sex,"female","tmp_female.csv",runlr_sex)
    df_bind <- rbind(df_both_stats,df_male_stats,df_female_stats)
    write.csv(df_bind, file=filename, row.name=TRUE)
    return(df_bind)
}

get_stats <- function(df,statfunc,sex, filename,lrfunc) {
    r2_stats <- boot(data=df, statistic=statfunc,R=10000, formula=phen~sex, parallel="multicore", ncpus=20)
    ci = boot.ci(r2_stats,type="bca")
    
    ste = sd(r2_stats$t)
    
    lr <- lrfunc(d=df)

    r2 <- r2_stats$t0 # average R2
    ci_lower = as.double(ci$bca[1,4])
    ci_upper = as.double(ci$bca[1,5])
    lr_pval = lr["Pr(>Chisq)"][2,1]

    df_stats <- data.frame("all_combined",sex,r2,ci_lower,ci_upper,lr_pval,ste) 

    # write.csv(df_stats, file=filename, row.name=TRUE)

    return(df_stats)
}

df <- read.table("all_t1burden_prs_200828.num.tsv", header = TRUE, sep = "\t") # both
dfm <- read.table("all_t1burden_prs_200828.num.male.tsv", header = TRUE, sep = "\t") # male
dff <- read.table("all_t1burden_prs_200828.num.female.tsv", header = TRUE, sep = "\t") # female

risk = "all_combined"
df_risk <- get_risk_stats(df_both=df,df_male=dfm,df_female=dff,paste(risk,"csv",sep="."))


In [ ]:
# prs_combined

library(fmsb)
library(lmtest)
library(boot)
options(stringsAsFactors = FALSE);

incremental_r2 <- function(formula, data, indices) {
    d <- data[indices,]
    fit <- glm(formula, data=d, family=binomial)
    fit_alt  <- glm(d$phen~d$sex+d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10+d$prs_asd+d$prs_ea+d$prs_int+d$prs_scz, data=d, family=binomial)
    fit_null <- glm(d$phen~d$sex+d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10, data=d, family=binomial)
    return(NagelkerkeR2(fit_alt)$R2 - NagelkerkeR2(fit_null)$R2)
}

incremental_r2_sex <- function(formula, data, indices) {
    d <- data[indices,]
    fit <- glm(formula, data=d, family=binomial)
    fit_alt  <- glm(d$phen~d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10+d$prs_asd+d$prs_ea+d$prs_int+d$prs_scz, data=d, family=binomial)
    fit_null <- glm(d$phen~d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10, data=d, family=binomial)
    return(NagelkerkeR2(fit_alt)$R2 - NagelkerkeR2(fit_null)$R2)
}

runlr <- function(d) {
    fit_alt  <- glm(d$phen~d$sex+d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10+d$prs_asd+d$prs_ea+d$prs_int+d$prs_scz, data=d, family=binomial)
    fit_null <- glm(d$phen~d$sex+d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10, data=d, family=binomial)
    return(lrtest(fit_alt,fit_null))   
}

runlr_sex <- function(d) {
    fit_alt  <- glm(d$phen~d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10+d$prs_asd+d$prs_ea+d$prs_int+d$prs_scz, data=d, family=binomial)
    fit_null <- glm(d$phen~d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10, data=d, family=binomial)
    return(lrtest(fit_alt,fit_null))   
}

get_risk_stats <-function(df_both,df_male,df_female,filename){
    df_both_stats <- get_stats(df=df_both,statfunc=incremental_r2,"both","tmp_both.csv",runlr)
    df_male_stats <- get_stats(df=df_male,statfunc=incremental_r2_sex,"male","tmp_male.csv",runlr_sex)
    df_female_stats <- get_stats(df=df_female,statfunc=incremental_r2_sex,"female","tmp_female.csv",runlr_sex)
    df_bind <- rbind(df_both_stats,df_male_stats,df_female_stats)
    write.csv(df_bind, file=filename, row.name=TRUE)
    return(df_bind)
}

get_stats <- function(df,statfunc,sex, filename,lrfunc) {
    r2_stats <- boot(data=df, statistic=statfunc,R=10000, formula=phen~sex, parallel="multicore", ncpus=20)
    ci = boot.ci(r2_stats,type="bca")
    
    ste = sd(r2_stats$t)
    
    lr <- lrfunc(d=df)

    r2 <- r2_stats$t0 # average R2
    ci_lower = as.double(ci$bca[1,4])
    ci_upper = as.double(ci$bca[1,5])
    lr_pval = lr["Pr(>Chisq)"][2,1]

    df_stats <- data.frame("prs_combined",sex,r2,ci_lower,ci_upper,lr_pval,ste) 

    # write.csv(df_stats, file=filename, row.name=TRUE)

    return(df_stats)
}


df <- read.table("all_t1burden_prs_200828.num.tsv", header = TRUE, sep = "\t") # both
dfm <- read.table("all_t1burden_prs_200828.num.male.tsv", header = TRUE, sep = "\t") # male
dff <- read.table("all_t1burden_prs_200828.num.female.tsv", header = TRUE, sep = "\t") # female

risk = "prs_combined"
df_risk <- get_risk_stats(df_both=df,df_male=dfm,df_female=dff,paste(risk,"csv",sep="."))


In [ ]:
# rare_combined

library(fmsb)
library(lmtest)
library(boot)
options(stringsAsFactors = FALSE);

incremental_r2 <- function(formula, data, indices) {
    d <- data[indices,]
    fit <- glm(formula, data=d, family=binomial)
    fit_alt  <- glm(d$phen~d$sex+d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10+d$snv_dn_lof+d$snv_dn_mis+d$snv_lof_t, data=d, family=binomial)
    fit_null <- glm(d$phen~d$sex+d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10, data=d, family=binomial)
    return(NagelkerkeR2(fit_alt)$R2 - NagelkerkeR2(fit_null)$R2)
}

incremental_r2_sex <- function(formula, data, indices) {
    d <- data[indices,]
    fit <- glm(formula, data=d, family=binomial)
    fit_alt  <- glm(d$phen~d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10+d$snv_dn_lof+d$snv_dn_mis+d$snv_lof_t, data=d, family=binomial)
    fit_null <- glm(d$phen~d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10, data=d, family=binomial)
    return(NagelkerkeR2(fit_alt)$R2 - NagelkerkeR2(fit_null)$R2)
}

runlr <- function(d) {
    fit_alt  <- glm(d$phen~d$sex+d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10+d$snv_dn_lof+d$snv_dn_mis+d$snv_lof_t, data=d, family=binomial)
    fit_null <- glm(d$phen~d$sex+d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10, data=d, family=binomial)
    return(lrtest(fit_alt,fit_null))   
}

runlr_sex <- function(d) {
    fit_alt  <- glm(d$phen~d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10+d$snv_dn_lof+d$snv_dn_mis+d$snv_lof_t, data=d, family=binomial)
    fit_null <- glm(d$phen~d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10, data=d, family=binomial)
    return(lrtest(fit_alt,fit_null))   
}

get_risk_stats <-function(df_both,df_male,df_female,filename){
    df_both_stats <- get_stats(df=df_both,statfunc=incremental_r2,"both","tmp_both.csv",runlr)
    df_male_stats <- get_stats(df=df_male,statfunc=incremental_r2_sex,"male","tmp_male.csv",runlr_sex)
    df_female_stats <- get_stats(df=df_female,statfunc=incremental_r2_sex,"female","tmp_female.csv",runlr_sex)
    df_bind <- rbind(df_both_stats,df_male_stats,df_female_stats)
    write.csv(df_bind, file=filename, row.name=TRUE)
    return(df_bind)
}

get_stats <- function(df,statfunc,sex, filename,lrfunc) {
    r2_stats <- boot(data=df, statistic=statfunc,R=10000, formula=phen~sex, parallel="multicore", ncpus=20)
    ci = boot.ci(r2_stats,type="bca")
    
    ste = sd(r2_stats$t)
    
    lr <- lrfunc(d=df)

    r2 <- r2_stats$t0 # average R2
    ci_lower = as.double(ci$bca[1,4])
    ci_upper = as.double(ci$bca[1,5])
    lr_pval = lr["Pr(>Chisq)"][2,1]

    df_stats <- data.frame("rare_combined",sex,r2,ci_lower,ci_upper,lr_pval,ste) 

    # write.csv(df_stats, file=filename, row.name=TRUE)

    return(df_stats)
}

df <- read.table("all_t1burden_prs_200828.num.tsv", header = TRUE, sep = "\t") # both
dfm <- read.table("all_t1burden_prs_200828.num.male.tsv", header = TRUE, sep = "\t") # male
dff <- read.table("all_t1burden_prs_200828.num.female.tsv", header = TRUE, sep = "\t") # female

risk = "rare_combined"
df_risk <- get_risk_stats(df_both=df,df_male=dfm,df_female=dff,paste(risk,"csv",sep="."))


In [ ]:
# individual risks R2

library(fmsb)
library(lmtest)
library(boot)
options(stringsAsFactors = FALSE);

incremental_r2 <- function(formula, data, indices, var) {
    d <- data[indices,]
    fit <- glm(formula, data=d, family=binomial)
    fit_alt  <- glm(d$phen~d$sex+d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10+d[[var]], data=d, family=binomial)
    fit_null <- glm(d$phen~d$sex+d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10, data=d, family=binomial)
    return(NagelkerkeR2(fit_alt)$R2 - NagelkerkeR2(fit_null)$R2)
}

incremental_r2_sex <- function(formula, data, indices, var) {
    d <- data[indices,]
    fit <- glm(formula, data=d, family=binomial)
    fit_alt  <- glm(d$phen~d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10+d[[var]], data=d, family=binomial)
    fit_null <- glm(d$phen~d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10, data=d, family=binomial)
    return(NagelkerkeR2(fit_alt)$R2 - NagelkerkeR2(fit_null)$R2)
}

runlr <- function(d, var) {
    fit_alt  <- glm(d$phen~d$sex+d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10+d[[var]], data=d, family=binomial)
    fit_null <- glm(d$phen~d$sex+d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10, data=d, family=binomial)
    return(lrtest(fit_alt,fit_null))   
}
# remove sex covariate (only males or females in the dataset)
runlr_sex <- function(d, var) {
    fit_alt  <- glm(d$phen~d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10+d[[var]], data=d, family=binomial)
    fit_null <- glm(d$phen~d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10, data=d, family=binomial)
    return(lrtest(fit_alt,fit_null))   
}

get_risk_stats <-function(df_both,df_male,df_female,var,filename){
    df_both_stats <- get_stats(df=df_both,var=var,statfunc=incremental_r2,"both","tmp_both.csv",lrfunc=runlr)
    df_male_stats <- get_stats(df=df_male,var=var,statfunc=incremental_r2_sex,"male","tmp_male.csv",lrfunc=runlr_sex)
    df_female_stats <- get_stats(df=df_female,var=var,statfunc=incremental_r2_sex,"female","tmp_female.csv",lrfunc=runlr_sex)
    df_bind <- rbind(df_both_stats,df_male_stats,df_female_stats)
    write.csv(df_bind, file=filename, row.name=TRUE)
    return(df_bind)
}

get_stats <- function(df,var,statfunc,sex, filename,lrfunc) {
    r2_stats <- boot(data=df, statistic=statfunc,R=10000, formula=phen~sex, var=var, parallel="multicore", ncpus=20)
    # r2_stats <- boot(data=df, statistic=statfunc,R=1200, formula=phen~sex, var=var, parallel="multicore", ncpus=20)
    ci = boot.ci(r2_stats,type="bca")
    
    ste = sd(r2_stats$t)

    # lr <- runlr(d=df,var=var)
    lr <- lrfunc(d=df,var=var)

    r2 <- r2_stats$t0 # average R2
    ci_lower = as.double(ci$bca[1,4])
    ci_upper = as.double(ci$bca[1,5])
    lr_pval = lr["Pr(>Chisq)"][2,1]

    df_stats <- data.frame(var,sex,r2,ci_lower,ci_upper,lr_pval,ste) 

    # write.csv(df_stats, file=filename, row.name=TRUE)

    return(df_stats)
}

df <- read.table("all_t1burden_prs_200828.num.tsv", header = TRUE, sep = "\t") # both
dfm <- read.table("all_t1burden_prs_200828.num.male.tsv", header = TRUE, sep = "\t") # male
dff <- read.table("all_t1burden_prs_200828.num.female.tsv", header = TRUE, sep = "\t") # female

# df <- read.table("all_t1burden_prs_200828.num.female.test1k.tsv", header = TRUE, sep = "\t") # both
# dfm <- read.table("all_t1burden_prs_200828.num.male.test1k.tsv", header = TRUE, sep = "\t") # male
# dff <- read.table("all_t1burden_prs_200828.num.female.test1k.tsv", header = TRUE, sep = "\t") # female


risk = "snv_dn_lof"
df_risk <- get_risk_stats(df_both=df,df_male=dfm,df_female=dff,var=risk,paste(risk,"csv",sep="."))
 
risk = "snv_dn_mis"
df_risk <- get_risk_stats(df_both=df,df_male=dfm,df_female=dff,var=risk,paste(risk,"csv",sep="."))

risk = "snv_lof_t"
df_risk <- get_risk_stats(df_both=df,df_male=dfm,df_female=dff,var=risk,paste(risk,"csv",sep="."))
 
risk = "ptdt_prs_asd"
df_risk <- get_risk_stats(df_both=df,df_male=dfm,df_female=dff,var=risk,paste(risk,"csv",sep="."))

risk = "ptdt_prs_ea"
df_risk <- get_risk_stats(df_both=df,df_male=dfm,df_female=dff,var=risk,paste(risk,"csv",sep="."))

risk = "prs_int_load"
df_risk <- get_risk_stats(df_both=df,df_male=dfm,df_female=dff,var=risk,paste(risk,"csv",sep="."))

risk = "ptdt_prs_scz"
df_risk <- get_risk_stats(df_both=df,df_male=dfm,df_female=dff,var=risk,paste(risk,"csv",sep="."))

risk = "prs_asd"
df_risk <- get_risk_stats(df_both=df,df_male=dfm,df_female=dff,var=risk,paste(risk,"csv",sep="."))

risk = "prs_ea"
df_risk <- get_risk_stats(df_both=df,df_male=dfm,df_female=dff,var=risk,paste(risk,"csv",sep="."))

risk = "prs_int"
df_risk <- get_risk_stats(df_both=df,df_male=dfm,df_female=dff,var=risk,paste(risk,"csv",sep="."))

risk = "prs_scz"
df_risk <- get_risk_stats(df_both=df,df_male=dfm,df_female=dff,var=risk,paste(risk,"csv",sep="."))


In [ ]:
# Responses from residuals
library(fmsb)
options(stringsAsFactors = FALSE);

d <- read.table("all_t1burden_prs_200828.num.iid.response.tsv", header = TRUE, sep = ",")

# PRS scores residualized:
ptdt_prs_asd_resid <- resid(glm(d$ptdt_prs_asd~d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10+d$sex, data=d,na.action="na.exclude"))
ptdt_prs_ea_resid <- resid(glm(d$ptdt_prs_ea~d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10+d$sex, data=d,na.action="na.exclude") )
ptdt_prs_scz_resid <- resid(glm(d$ptdt_prs_scz~d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10+d$sex, data=d,na.action="na.exclude") )

# Fit combined prs model
fit_alt_prs <- glm(d$phen~ptdt_prs_asd_resid+ptdt_prs_ea_resid+ptdt_prs_scz_resid, data=d, family=binomial)
response_prs = predict(fit_alt_prs, type = "response")

# rare scores residualized
snv_dn_lof_resid <- resid(glm(d$snv_dn_lof~d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10+d$sex, data=d,na.action="na.exclude") )
snv_dn_mis_resid <- resid(glm(d$snv_dn_mis~d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10+d$sex, data=d,na.action="na.exclude") )
snv_lof_t_resid <- resid(glm(d$snv_lof_t~d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10+d$sex, data=d,na.action="na.exclude") )

# Fit combined rare model
fit_alt_rare <- glm(d$phen~snv_dn_lof_resid+snv_dn_mis_resid+snv_lof_t_resid, data=d, family=binomial)
rare_combined_score = predict(fit_alt_rare, type = "response")

# Fit rare + prs combined model
fit_alt_combined <- glm(d$phen~ptdt_prs_asd_resid+ptdt_prs_ea_resid+ptdt_prs_scz_resid+snv_dn_lof_resid+snv_dn_mis_resid+snv_lof_t_resid, data=d, family=binomial)
all_combined_score = predict(fit_alt_combined, type = "response")

# Fit individual features
# ptdt_prs_asd
fit_alt_ptdt_prs_asd <- glm(d$phen~ptdt_prs_asd_resid, data=d, family=binomial)
response_ptdt_prs_asd = predict(fit_alt_ptdt_prs_asd, type = "response")
# ptdt_prs_ea
fit_alt_ptdt_prs_ea <- glm(d$phen~ptdt_prs_ea_resid, data=d, family=binomial)
response_ptdt_prs_ea = predict(fit_alt_ptdt_prs_ea, type = "response")
# ptdt_prs_scz
fit_alt_ptdt_prs_scz <- glm(d$phen~ptdt_prs_scz_resid, data=d, family=binomial)
response_ptdt_prs_scz = predict(fit_alt_ptdt_prs_scz, type = "response")

# snv_dn_lof
fit_alt_snv_dn_lof <- glm(d$phen~snv_dn_lof_resid, data=d, family=binomial)
response_snv_dn_lof = predict(fit_alt_snv_dn_lof, type = "response")
# snv_dn_mis
fit_alt_snv_dn_mis <- glm(d$phen~snv_dn_mis_resid, data=d, family=binomial)
response_snv_dn_mis = predict(fit_alt_snv_dn_mis, type = "response")
# snv_lof_t
fit_alt_snv_lof_t <- glm(d$phen~snv_lof_t_resid, data=d, family=binomial)
response_snv_lof_t = predict(fit_alt_snv_lof_t, type = "response")

d$all_combined_score = all_combined_score
d$ps_combined_score = response_prs
d$rare_combined_score = rare_combined_score
d$response_ptdt_prs_asd = response_ptdt_prs_asd
d$response_ptdt_prs_asd = response_ptdt_prs_ea
d$response_ptdt_prs_scz = response_ptdt_prs_scz
d$response_snv_dn_lof = response_snv_dn_lof
d$response_snv_dn_mis = response_snv_dn_mis
d$response_snv_lof_t = response_snv_lof_t

write.csv(d, file="all_t1burden_prs_200828.num.iid.responses_from_residuals.20201209.csv", row.name=TRUE)


<a name="paternal_maternal_age_correlations"></a>
## Paternal and Maternal Age Correlations with De Novo Mutations

In [ ]:
# Paternal and maternal age correlation

df_dnms_ages.autosomal.noncoding <- read.delim("df_dnms_ages.autosomal.noncoding.tsv")
lmDNM = lm(dnm_count ~ father_age + mother_age, data = df_dnms_ages.autosomal.noncoding)
summary(lmDNM)


dnms_ages_vars <- c("dnm_count","father_age","mother_age")
dnms_ages_df <- df_dnms_ages.autosomal.noncoding[dnms_ages_vars]
corrmat <- cor(dnms_ages_df, method = "pearson", use = "complete.obs")
corrmat



Call:
lm(formula = dnm_count ~ father_age + mother_age, data = df_dnms_ages.autosomal.noncoding)

Residuals:
    Min      1Q  Median      3Q     Max 
-31.943  -6.997  -0.548   6.324 128.131 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) 15.08212    1.06375  14.178  < 2e-16 ***
father_age   1.29981    0.04083  31.833  < 2e-16 ***
mother_age   0.29692    0.04567   6.502  8.8e-11 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 10.6 on 4505 degrees of freedom
Multiple R-squared:  0.3783,	Adjusted R-squared:  0.378 
F-statistic:  1371 on 2 and 4505 DF,  p-value: < 2.2e-16


,dnm_count,father_age,mother_age
dnm_count,1.0000000,0.6103209,0.4883549
father_age,0.6103209,1.0000000,0.7123290
mother_age,0.4883549,0.7123290,1.0000000


In [ ]:
lmDNM_father = lm(dnm_count ~ father_age, data = df_dnms_ages.autosomal.noncoding)
summary(lmDNM_father)
lmDNM_mother = lm(dnm_count ~ mother_age, data = df_dnms_ages.autosomal.noncoding)
summary(lmDNM_mother)


Call:
lm(formula = dnm_count ~ father_age, data = df_dnms_ages.autosomal.noncoding)

Residuals:
    Min      1Q  Median      3Q     Max 
-35.582  -7.030  -0.495   6.402 127.979 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) 18.03250    0.96650   18.66   <2e-16 ***
father_age   1.48892    0.02879   51.72   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 10.65 on 4506 degrees of freedom
Multiple R-squared:  0.3725,	Adjusted R-squared:  0.3724 
F-statistic:  2675 on 1 and 4506 DF,  p-value: < 2.2e-16



Call:
lm(formula = dnm_count ~ mother_age, data = df_dnms_ages.autosomal.noncoding)

Residuals:
    Min      1Q  Median      3Q     Max 
-35.407  -7.856  -0.859   6.919 129.811 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) 25.99596    1.11438   23.33   <2e-16 ***
mother_age   1.33242    0.03547   37.57   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 11.74 on 4506 degrees of freedom
Multiple R-squared:  0.2385,	Adjusted R-squared:  0.2383 
F-statistic:  1411 on 1 and 4506 DF,  p-value: < 2.2e-16
